In [57]:
import os
import pandas as pd
from openai import OpenAI


# Read data

In [58]:
#train_folder = "dataset/AdMIRe Subtask A Train/train/"
#train_df_file_path = train_folder + "subtask_a_train.tsv"

# train_folder = "dataset/AdMIRe Subtask A PT Train/"
# train_df_file_path = train_folder + "subtask_a_train.tsv"

#train_folder = "dataset/AdMIRe Subtask A Dev/dev/"
#train_df_file_path = train_folder + "subtask_a_dev.tsv"

#train_folder = "dataset/AdMIRe Subtask A PT Dev/dev/"
#train_df_file_path = train_folder + "subtask_a_dev.tsv"

# train_folder = "dataset/AdMIRe Subtask A Test/"
# train_df_file_path = train_folder + "subtask_a_test.tsv"

train_folder = "dataset/AdMIRe Subtask A PT Test/"
train_df_file_path = train_folder + "subtask_a_test.tsv"

df = pd.read_csv(train_df_file_path, sep='\t')


In [59]:
df

,compound,subset,sentence_type,sentence,expected_order,image1_name,image1_caption,image1_caption_pt,image2_name,image2_caption,image2_caption_pt,image3_name,image3_caption,image3_caption_pt,image4_name,image4_caption,image4_caption_pt,image5_name,image5_caption,image5_caption_pt
0,caixa preta,Test,NaN,Fui à loja comprar uma caixa colorida para dec...,NaN,02617555490.png,The image shows a vintage-style suitcase with ...,A imagem mostra uma maleta de viagem antiga co...,04933761386.png,The image shows an illustration of a man sitti...,A imagem mostra um homem sentado em uma cadeir...,67551849161.png,The image shows a stylized illustration of a w...,A imagem mostra uma ilustração de uma mulher s...,74286981889.png,The image shows an illustration of a man holdi...,A imagem mostra um homem segurando uma grande ...,78074168989.png,The image shows a man holding a tablet with a ...,A imagem mostra um homem sorridente segurando ...
1,água doce,Test,NaN,Muita gente toma água com açúcar pra se acalma...,NaN,35657964996.png,"The image shows an animated character, a man w...","A imagem mostra um personagem animado, que par...",52906318587.png,The image shows a hand holding a small piece o...,A imagem mostra uma mão de uma pessoa pegando ...,60718356798.png,"The image shows an animated character, a bald ...","A imagem mostra um personagem animado, que par...",78610302321.png,"The image shows an animated character, a young...",A imagem mostra um jovem em um ambiente natura...,93704824032.png,"The image shows an animated character, a young...",A imagem mostra um desenho de um pequeno menin...
2,algodão doce,Test,NaN,Esta máquina de algodão doce profissional é pe...,NaN,02435667093.png,The image shows an illustration of a young boy...,A imagem mostra um menino caminhando através d...,52781925731.png,The image shows a box containing what appears ...,A imagem mostra uma caixa de papelão com seis ...,58070129781.png,The image shows a cartoon-style illustration o...,A imagem mostra um desenho de um menino sorrid...,59437741462.png,"The image shows a large, fluffy white cloud ag...","A imagem mostra um grande nublado branco, que ...",89608564565.png,"The image shows a chocolate cake with a rich, ...",A imagem mostra um bolo decorado com uma varie...
3,alto mar,Test,NaN,A maré subiu enquanto o menino nadava e o alto...,NaN,06187252072.png,The image shows a digital rendering of a ship ...,A imagem mostra um cenário marinho com um navi...,24004322677.png,The image shows a stylized illustration of a m...,A imagem mostra um ilustração de um homem posa...,39706172854.png,The image shows a stylized illustration of a s...,A imagem mostra uma representação estilizada d...,48227996732.png,The image depicts a stylized and exaggerated r...,A imagem mostra uma ilustração de uma onda gig...,84134178486.png,The image depicts a lively scene at a beach. T...,A imagem mostra uma cena de banho em uma praia...
4,tapete vermelho,Test,NaN,"Watts afirmou em entrevista, em um tapete verm...",NaN,10431181796.png,The image depicts a group of stylized characte...,A imagem mostra uma ilustração de uma festa ou...,13926060291.png,The image shows a cute kitten with a mix of wh...,A imagem mostra um gato recolhido e adormecido...,46251467125.png,"The image shows two individuals, likely at a p...",A imagem mostra duas mulheres em um ambiente q...,50992610818.png,The image shows a modern living room setting. ...,A imagem mostra um ambiente de interior decora...,58378881138.png,The image shows an animated depiction of a gol...,A imagem mostra um cachorro cor-de-laranja em ...
5,planta baixa,Test,NaN,Comprei dois tipos de planta para o meu jardi...,NaN,21109433823.png,The image depicts an illustration of a man and...,A imagem mostra dois personagens sentados em f...,28962122356.png,The image shows an illustration of two animate...,A imagem mostra dois personagens animados em u...,41754702570.png,"The image shows an animated character, a woman...",A imagem mostra dois personagens ani

# Define a function to get an LLM response

In [60]:
api_key = "" # <=========================== Change the API key here

client = OpenAI(
    api_key=api_key,
)

# Define the model name dynamically
model_name = "gpt-4o"

def determine_sentence_type(message):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": "You are a professional linguist specializing in figurative language. You will be given a compound and a sentence containing this compound. Your task is determining whether this compound has an idiomatic or literal meaning in the sentence."
          },
          {
              "role": "user",
              "content": message,
          }
      ],
      model=model_name,
  )
  response_content = chat_completion.choices[0].message.content
  return response_content

def generate_idiom_meaning(message):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": "You are a professional linguist specializing in figurative language. You will be given an idiom. Your task is generating its meaning without any accompanying texts."
          },
          {
              "role": "user",
              "content": message,
          }
      ],
      model=model_name,
  )
  response_content = chat_completion.choices[0].message.content
  return response_content

# Generate idomatic meanings

In [61]:
meanings = []
sent_types = []
count = 0

from collections import Counter

def get_majority_response(query, n=5):
    responses = []
    for _ in range(n):
        response = determine_sentence_type(query).strip()
        if response in ["Literal", "Idiomatic"]:
            responses.append(response)
    print(responses)
    most_common_response, _ = Counter(responses).most_common(1)[0]
    return most_common_response

# Check if 'sentence_type' exists in the dataframe
has_sentence_type = 'sentence_type' in df.columns

for _, row in df.iterrows():
    compound = row['compound']
    sent_type = row['sentence_type']
    sent = row['sentence']

    # If 'sentence_type' exists, use it
    sent_type = row['sentence_type'] if has_sentence_type else None

    print(compound)
    print(sent)

    # First determine if the compound has an idiomatic or literal meaning based on a given sentence
    query = f"Given an example sentence: {sent}\nIs the phrase \"{compound}\" has literal or idiomatic meaning?\n" + "Only answer \"Literal\" or \"Idiomatic\"."
    response = get_majority_response(query, n=5)
    if sent_type == "idiomatic" and response.lower() == "idiomatic":
        count+=1
    elif sent_type == "literal" and response.lower() == "literal":
        count+=1
    print(sent_type, response.lower())

    # If it has an idiomatic meaning, then generate its meaning
    if response.lower() == 'idiomatic':
        query = f"Given an example sentence: {sent}\nWhat are the idiomatic meanings of {compound} in this example sentence?\n" + "Provide one answer wihtout accompanying texts."
        response = generate_idiom_meaning(query)
        print("idiomatic:", response)
        response.replace("\\", '').replace("'", "\\'").replace("\\\\'", "\\'")
        meaning = response
        meanings.append(meaning)
        sent_types.append("idiomatic")
    elif response.lower() == 'literal':
        meanings.append(None)
        sent_types.append("literal")
    else:
        raise NotImplemetedError

    print('')

# Save the generated meanings
df["meaning"] = meanings

# Save the determined compound type (idiomatic or literal)
if all(df["sentence_type"].isna()):
    df["sentence_type"] = sent_types

df["sent_type_predicted"] = sent_types

# Save data with a dynamically generated file name
output_file_name = train_df_file_path.replace(".tsv", f"_{model_name}_meanings.tsv")
df.to_csv(output_file_name, sep='\t', index=False)

caixa preta
Fui à loja comprar uma caixa colorida para decorar o escritório, mas só tinha caixa preta.
['Literal', 'Literal', 'Literal', 'Literal', 'Literal']
nan literal

água doce
Muita gente toma água com açúcar pra se acalmar, mas por causa do diabetes eu não posso ingerir água doce.
['Literal', 'Literal', 'Literal', 'Literal', 'Literal']
nan literal

algodão doce
Esta máquina de algodão doce profissional é perfeita para eventos e festas.
['Literal', 'Literal', 'Literal', 'Literal', 'Literal']
nan literal

alto mar
A maré subiu enquanto o menino nadava e o alto ar o deixou submerso.
['Idiomatic', 'Idiomatic', 'Literal', 'Literal', 'Literal']
nan literal

tapete vermelho
Watts afirmou em entrevista, em um tapete vermelho pouco concorrido, que estava preocupada com o que os filhos de Diana, os príncipes William e Harry, iriam pensar de o filme caso o vejam.
['Idiomatic', 'Idiomatic', 'Idiomatic', 'Idiomatic', 'Idiomatic']
nan idiomatic
idiomatic: "Tapete vermelho" in this context idi

In [62]:
# print(count/len(df))